# PINNs: Physics-Informed Neural Networks
@Adil Thami (2024)

Sources:
- Goodfellow et al. (2016), Deep Learning [(Book)](https://www.deeplearningbook.org/)
- Raissi et al. (2019), Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations [(Paper)](https://www.sciencedirect.com/science/article/pii/S0021999118307125)
- Moseley, B. (2023). Physics-informed neural network (PINN) demo [(Notebook)](https://github.com/benmoseley/DLSC-2023/blob/main/lecture-5/PINN%20demo.ipynb)
- Damped Harmonic Oscillator - Derivation and solution of the differential equations [(Beltoforion.de)](https://beltoforion.de/en/harmonic_oscillator/)

In [1]:
import os
import pandas as pd
import numpy as np
'''
    PyTorch: libraries for deep learning.
    - Provide good integration with NumPy (Uuseful for data preprocessing and visualization) and automatic differentiation (useful for training models).
    - Well known for scientific computing.

    NB: everything can also be done in TensorFlow (or other frameworks).
'''
import torch
import torch.nn as nn
import torch.nn.init as init
'''
    Plotly: library for interactive plots.
    - Useful for complex plots and dashboards or web applications.
    - Highly customizable.
'''
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default='notebook'
'''
    Scikit-learn: library for machine learning.
    - Provide tools for data preprocessing, model selection, evaluation and more.
    - Well known for its ease of use.
'''
from sklearn.metrics import r2_score

import IPython.display as display

# gpu or cpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## 1. Time dependent ODE
### 1.1. Case study: Damped Harmonic Oscillator

The equation of motion for a damped harmonic oscillator is given by a second order differential equation of the form:

$$ 
m \frac{d^2 x}{dt^2} + \mu \frac{dx}{dt} + k x = 0
$$

where $m$ is the mass $[M]$, $\mu$ is the damping/friction coefficient $[MT^{-1}]$, and $k$ is the spring constant $[MT^{-2}]$.

If the ***under-damped*** case is considered, *i.e*, when the oscillation is damped by a friction force, the displacement $x(t) ~ [L]$ as a function of time $t$ $[T]$ can be written as:

\begin{align*}
x(t) = e^{-\delta t}(2 A \cos(\phi + \omega t))
\end{align*}

where $\delta$ is the damping coefficient $[MT^{-1}]$, $\omega$ and $\omega_0$ are the angular frequency and natural frequency $[T^{-1}]$, and $A$ $[L]$ and $\phi$ $[-]$ are the amplitude and phase of the oscillation.

\begin{align*}
\omega &=\sqrt{\omega_0^2 - \delta^2}\\

\omega_0 &= \sqrt{\frac{k}{m}} \\
\delta &= \frac{\mu}{2m} ~~ < \omega_0 \\

\end{align*}

Note that the exponential term $e^{−\delta t}$ significantly reduces the amplitude of oscillation if $\delta$ is large.

In [2]:
def exact_solution(delta, w0, t):
    ''' 
        Defines the analytical solution to the under-damped harmonic oscillator problem.
    '''
    assert delta < w0
    w = np.sqrt(w0**2-delta**2)
    phi = np.arctan(-delta/w)
    A = 1/(2*np.cos(phi))
    cos = torch.cos(phi+w*t)
    exp = torch.exp(-delta*t)
    u = exp*2*A*cos
    return u

delta, w0 = 2, 20
mu, k = 2*delta, w0**2
t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(delta, w0, t_test)

In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0], y=[exact_solution(delta, w0, t_test)[0].item()], mode='markers', name='Mass position', marker=dict(size=20, color='orange')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=exact_solution(delta, w0, t_test).flatten(), mode='lines', name='Exact solution', line=dict(color='lightblue')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=torch.zeros_like(t_test).flatten(), mode='lines', name='Equilibrium position', line=dict(color='red', dash='dash')))
fig.update_layout(title='Exact solution of the under-damped harmonic oscillator', xaxis_title='Time (s)', yaxis_title='Displacement (m)')
fig.update_yaxes(range=[-1,1])
fig.update_xaxes(range=[0,1])
fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration='1', redraw=True), fromcurrent=True)])])])
frames = [go.Frame(data=[go.Scatter(x=[t], y=[u], mode='markers')], name=str(t)) for t, u in zip(t_test.flatten(), u_exact.flatten())]
fig.frames = frames
fig.update_layout(template='plotly_dark')
# add a gif (Damped_speed.gif) to the plot below the legend
fig.show()

### 1.2. PINN - Forward problem

The goal is to approximate the exact solution of the damped harmonic oscillator for a given set of parameters $m$, $\mu$, and $k$ and the general equation of the ODE.

#### 1.2.1. Neural network architecture

In general, a PINN is composed of a Feed-Forward Neural Network (FNN) that approximates the solution of the ODE/PDE and a loss function that enforces the ODE/PDE constraints.

The FNN can be represented as:

<img src="NN_schema.svg" width="900" />

Here we consider a network with the following architecture:
- 3 hidden layers
- 30 neurons per hidden layer
- Tanh activation function


In [4]:
# Define the neural network
class NeuralNet(nn.Module):
    '''
        The model is defined as a sequence of layers (nn.Sequential).
        The input layer has one neuron (t) and the output layer has one neuron (x(t)).
        The hidden layers have 30 neurons each and use the hyperbolic tangent activation function.
        The weights of the model are initialized using the Xavier uniform initialization.
    '''
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 30),  # Input layer (t) to hidden layer
            nn.Tanh(),         # Activation function
            nn.Linear(30, 30), # Hidden layer to hidden layer
            nn.Tanh(),         # Activation function
            nn.Linear(30, 30), # Hidden layer to hidden layer
            nn.Tanh(),         # Activation function
            nn.Linear(30, 30), # Hidden layer to hidden layer
            nn.Tanh(),         # Activation function
            nn.Linear(30, 1)   # Hidden layer to output layer (x(t))
        )
        self._init_weights()

    def _init_weights(self):
        for module in self.net:
            if isinstance(module, nn.Linear):
                # Apply Xavier uniform initialization to the weights of each Linear layer
                init.xavier_uniform_(module.weight)
                # Optionally, you can also set biases to 0 if they exist
                if module.bias is not None:
                    init.constant_(module.bias, 0)
    def forward(self, t):
        return self.net(t)

# Initialize the model
model = NeuralNet()

#### 1.2.2. How does it learn?

The outputs of the PINN are the terms of the solution of the ODE/PDE. 
This is done by ***automatic differentiation***, *i.e*, the gradients of the output with respect to the input data are computed using the chain rule of derivatives.

The neural network is trained by minimizing the loss function. The loss function is composed of two terms:
- Usually: The ***mean squared error*** (MSE) between the predicted solution and the true solution.

For a PINN:
- The ***physics-informed loss*** term that enforces the ODE/PDE constraints.
    - Minimize the residual of the ODE/PDE at a set of collocation points (x', t')
    - Minimize the residual of the initial and boundary conditions. (if applicable or known)

A PINN can be represented as:

<img src="PINN_schema.svg" width="900" />



In [5]:
# Define the physics-informed loss
def PINN(t_pred, t_boundary, model):
    ''' 
        - t_pred: the time points where the model will be evaluated
        - t_boundary: the time points where the boundary conditions are enforced
        - model: the neural network model

        Be aware of the size of the tensors. 
        - The model expects an input tensor of shape (n, 1), where n is the number of samples.
        - The output tensor has shape (n, 1)
        - The time tensors t_pred and t_boundary have shape (n, 1)
        - The boundary conditions are enforced at the initial time t0, so t_boundary has only one element, shape (1, 1).

        The function returns the model predictions, the total loss, the differential equation loss, and the initial condition loss.
    '''

    # Compute the predictions
    x_pred = model(t_pred)
    x_boundary = model(t_boundary)

    # Compute the gradients for each output component with respect to t, manually
    dxdt_pred = torch.autograd.grad(x_pred, t_pred, torch.ones_like(x_pred), create_graph=True)[0]
    d2xdt2_pred = torch.autograd.grad(dxdt_pred, t_pred, torch.ones_like(dxdt_pred), create_graph=True)[0]
    
    dxdt_pred_t0 = torch.autograd.grad(x_boundary, t_boundary, torch.ones_like(x_boundary), create_graph=True)[0]

    # Constants
    m = 1.0         # mass of the object
    mu = 4.0        # damping coefficient = 2*mass*drag coefficient
    k = 20.0**2     # spring stiffness constant

    # Residuals for the differential equation
    residual = m*d2xdt2_pred + mu*dxdt_pred + k*x_pred 
    
    # Loss for the differential equation residuals: Physics-informed loss
    loss_physic = torch.mean((residual)**2)
    
    # Initial condition loss
    u_t0 = 1.0      # initial displacement
    dudt_t0 = 0.0   # initial velocity
    loss_ic = ((torch.squeeze(x_boundary) - u_t0)**2 + (torch.squeeze(dxdt_pred_t0) - dudt_t0)**2)

    # hyperparameters for the loss
    lambda1 = 1e-4
    lambda2 = 1e-1
    
    # Total loss is the sum of the differential equation loss and the initial condition loss
    loss = lambda1*loss_physic + lambda2*loss_ic
    return x_pred, loss, loss_physic, loss_ic

#### 1.2.3. Training the PINN

***The training process*** is composed of the following steps:
1. Initialize the weights and biases of the neural network.
2. Forward pass: Compute the output of the neural network.
3. Compute the loss function.
4. Backward pass: Compute the gradients of the loss function with respect to the weights and biases.
5. Update the weights and biases using the optimizer.

    - The ***Adam*** (adaptive moment estimation) is used instead of the classical stochastic gradient descent (SGD) to minimize the loss function. It is an adaptive learning rate optimization algorithm that computes individual adaptive learning rates for different parameters.
    - The learning rate is a hyperparameter that controls how much the weights and biases of the neural network are updated during training.
    
6. Repeat steps 2-5 for a number of epochs.


In [6]:
# Prepare the training data (time points)
ti = 0                      # initial time
tf = 1                      # final time
nt = 300                    # number of time points
t_train = torch.linspace(ti, tf, nt).reshape(-1, 1).requires_grad_(True)
t_boundary = torch.tensor(0.).view(-1,1).requires_grad_(True)# (1, 1)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 15001 # number of training iterations per batch of data

# Lists to store the losses and model predictions
losses = []
losses_physic = []
losses_ic = []
results = []

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    x_pred, loss, loss_physic, loss_ic = PINN(t_train, t_boundary, model)
    loss.backward()  
    optimizer.step()

    losses.append(loss.item()), losses_physic.append(loss_physic.item()), losses_ic.append(loss_ic.item())
    results.append(model(t_train).detach().numpy())

    if epoch % 500 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.16607972979545593
Epoch 500, Loss: 0.09643206000328064
Epoch 1000, Loss: 0.09597984701395035
Epoch 1500, Loss: 0.07161650061607361
Epoch 2000, Loss: 0.06525193899869919
Epoch 2500, Loss: 0.06075394153594971
Epoch 3000, Loss: 0.04956430941820145
Epoch 3500, Loss: 0.037791598588228226
Epoch 4000, Loss: 0.02468612790107727
Epoch 4500, Loss: 0.020387951284646988
Epoch 5000, Loss: 0.018549256026744843
Epoch 5500, Loss: 0.016637258231639862
Epoch 6000, Loss: 0.013320205733180046
Epoch 6500, Loss: 0.01095467247068882
Epoch 7000, Loss: 0.009740239009261131
Epoch 7500, Loss: 0.008297843858599663
Epoch 8000, Loss: 0.007560809142887592
Epoch 8500, Loss: 0.007270669564604759
Epoch 9000, Loss: 0.006449597422033548
Epoch 9500, Loss: 0.003275745315477252
Epoch 10000, Loss: 0.0017118232790380716
Epoch 10500, Loss: 0.0007362905307672918
Epoch 11000, Loss: 0.00035433692391961813
Epoch 11500, Loss: 0.00022314010129775852
Epoch 12000, Loss: 0.0001632405910640955
Epoch 12500, Loss: 0.00012

Display the results of the training process.

In [14]:
last_results = results[-1]
last_results

# plot the last results
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_test.flatten(), y=last_results.flatten(), mode='lines', name='PINN solution', line=dict(color='blue', width=10)))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=torch.zeros_like(t_test).flatten(), mode='lines', name='Equilibrium position', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=exact_solution(delta, w0, t_test).flatten(), mode='lines', name='Exact solution', line=dict(color='lightblue')))
fig.update_layout(title='PINN solution of the under-damped harmonic oscillator', xaxis_title='Time (s)', yaxis_title='Displacement (m)')
fig.update_yaxes(range=[-1,1])
fig.update_xaxes(range=[0,1])
fig.update_layout(template='plotly_dark')
fig.show()

# R² score
print('R²=', r2_score(exact_solution(delta, w0, t_test).flatten(), last_results.flatten()))

R²= 0.9999785243539182


Plot the loss during each epoch.

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=losses, mode='lines', name='Total loss', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=np.arange(epochs), y=losses_physic, mode='lines', name='Physics loss', line=dict(color='red')))
fig.add_trace(go.Scatter(x=np.arange(epochs), y=losses_ic, mode='lines', name='Initial condition loss', line=dict(color='seagreen')))
fig.update_layout(title='Losses during training', xaxis_title='Epoch', yaxis_title='Loss', template='plotly_dark')
fig.show()

### 1.3. PINN - Inverse problem

Inverse modeling is the process of estimating the parameters of a model given the solution of the ODE/PDE. The solution could be obtained from experimental data or from a forward model.

In this case study, the inverse problem consists of finding the parameters $\mu$ of the damped harmonic oscillator given the solution of the ODE.

#### 1.3.1. Create synthetic data for training

- $\mu$ = 8

In [22]:
delta, w0 = 4, 15
mu, k = 2*delta, w0**2

# Generate the exact solution
t_test = torch.linspace(0,1,300).view(-1,1)
x_exact = exact_solution(delta, w0, t_test)

# Generate noisy data points for training (50 points)
t_data = torch.rand(50).view(-1,1)
x_data = exact_solution(delta, w0, t_data) + 0.05*torch.randn_like(t_data)

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0], y=[exact_solution(delta, w0, t_test)[0].item()], mode='markers', name='Mass position', marker=dict(size=20, color='orange')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=exact_solution(delta, w0, t_test).flatten(), mode='lines', name='Exact solution', line=dict(color='lightblue')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=torch.zeros_like(t_test).flatten(), mode='lines', name='Equilibrium position', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(x=t_data.flatten(), y=x_data.flatten(), mode='markers', name='Data points', marker=dict(size=10, color='lightblue')))
fig.update_layout(title='Exact solution of the under-damped harmonic oscillator and noisy data', xaxis_title='Time (s)', yaxis_title='Displacement (m)')
fig.update_yaxes(range=[-1,1])
fig.update_xaxes(range=[0,1])
fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=1, redraw=True), fromcurrent=True)])])])
fig.update_layout(template='plotly_dark')
frames = [go.Frame(data=[go.Scatter(x=[t], y=[u], mode='markers')], name=str(t)) for t, u in zip(t_test.flatten(), x_exact.flatten())]
fig.frames = frames
fig.show()

# Based on the exact solution, make an animation of the displacement solely on a vertical line
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0], y=[exact_solution(delta, w0, t_test)[0].item()], mode='markers', name='Mass position', marker=dict(size=20, color='orange')))
# fig.add_trace(go.Scatter(x=t_test.flatten(), y=exact_solution(delta, w0, t_test).flatten(), mode='lines', name='Exact solution', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=t_test.flatten(), y=torch.zeros_like(t_test).flatten(), mode='lines', name='Equilibrium position', line=dict(color='red', dash='dash')))
fig.update_layout(title='Exact solution of the under-damped harmonic oscillator', xaxis_title='', yaxis_title='Displacement (m)')
# remove x-axis
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(range=[-1,1])
fig.update_xaxes(range=[-1,1])
fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=1, redraw=True), fromcurrent=True)])])])
fig.update_layout(template='plotly_dark')
frames = [go.Frame(data=[go.Scatter(x=[0], y=[u], mode='markers')], name=str(u)) for u in x_exact.flatten()]
fig.frames = frames
fig.show()

#### 1.3.2. Modify the PINN architecture

When doing inversion, the parameters of the ODE that are considered for inversion are additional parameters to the neural network.

PINNs also allow to approximate the parameters of the ODE/PDE ***regardless of the knowledge of the boundary conditions.***


In [17]:
# Define the physics-informed loss
def PINN_Inv(t_pred, t_data, obs, par, model):
    ''' 
        - t_pred: the time points where the model will be evaluated
        - t_data: the time points where the boundary conditions are enforced
        - obs: the observed data points
        - par: the parameters to be estimated
        - model: the neural network model

        Be aware of the size of the tensors. 
        - The model expects an input tensor of shape (n, 1), where n is the number of samples (different or not for prediction and data).
        - The output tensor has shape (n, 1)
        - The time tensors t_pred and t_data have shape (n, 1)
        - The boundary conditions are enforced at the initial time t0, so t_boundary has only one element, shape (1, 1).

        The function returns the model predictions, the total loss, the differential equation loss, and the initial condition loss.
    '''
    # Compute the predictions
    x_pred = model(t_pred)

    # Compute the gradients for each output component with respect to t, manually
    dxdt_pred = torch.autograd.grad(x_pred, t_pred, torch.ones_like(x_pred), create_graph=True)[0]
    d2xdt2_pred = torch.autograd.grad(dxdt_pred, t_pred, torch.ones_like(dxdt_pred), create_graph=True)[0]
    
    # Constants
    m = 1.0         # mass of the object
    k = 15.0**2     # spring stiffness constant

    # Residuals for the differential equation
    mu_star = par[0]
    residual = m*d2xdt2_pred + mu_star*dxdt_pred + k*x_pred 
    'mu_star instead of mu !!!'
    
    # Loss for the differential equation residuals: Physics-informed loss
    loss_physic = torch.mean((residual)**2)
    
    # Data loss
    x_obs = obs
    x_data = model(t_data)

    loss_data = torch.mean((x_obs - x_data)**2)

    # hyperparameters for the loss
    lambda1 = 1
    lambda2 = 1e4
    
    # Total loss is the sum of the differential equation loss and the initial condition loss
    loss = lambda1*loss_physic + lambda2*loss_data
    return x_pred, loss, loss_physic, loss_data

In [18]:
# record mu value
# Prepare the training data (time points)
ti = 0                      # initial time
tf = 1                    # final time
nt = 30                    # number of time points
t_train = torch.linspace(ti, tf, nt).reshape(-1, 1).requires_grad_(True)
obs = x_data

# Training setup
mu_star = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
optimizer = torch.optim.Adam(list(model.parameters())+[mu_star], lr=1e-3)
epochs = 15001 # number of training iterations per batch of data

# Lists to store the losses and model predictions
losses = []
losses_physic = []
losses_data = []
results = []
mus = []

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    x_pred, loss, loss_physic, loss_data = PINN_Inv(t_train, t_data, obs,[mu_star], model)
    loss.backward()  
    optimizer.step()
    losses.append(loss.item()), losses_physic.append(loss_physic.item()), losses_data.append(loss_data.item())
    results.append(model(t_train).detach().numpy())
    mus.append(mu_star.item())
    if epoch % 500 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, mu_star: {mu_star.item()}')

Epoch 0, Loss: 5232.40087890625, mu_star: 0.0009999999310821295
Epoch 500, Loss: 212.98162841796875, mu_star: 0.36061573028564453
Epoch 1000, Loss: 190.3931121826172, mu_star: 0.8038269281387329
Epoch 1500, Loss: 167.94483947753906, mu_star: 1.2644233703613281
Epoch 2000, Loss: 148.3896942138672, mu_star: 1.7241649627685547
Epoch 2500, Loss: 127.19384765625, mu_star: 2.175795078277588
Epoch 3000, Loss: 109.75041198730469, mu_star: 2.6159636974334717
Epoch 3500, Loss: 94.2427978515625, mu_star: 3.043283224105835
Epoch 4000, Loss: 80.82404327392578, mu_star: 3.457257032394409
Epoch 4500, Loss: 69.25956726074219, mu_star: 3.858027696609497
Epoch 5000, Loss: 60.30094528198242, mu_star: 4.245784282684326
Epoch 5500, Loss: 51.16716003417969, mu_star: 4.620912551879883
Epoch 6000, Loss: 44.26929473876953, mu_star: 4.983503341674805
Epoch 6500, Loss: 38.56401824951172, mu_star: 5.33348274230957
Epoch 7000, Loss: 33.83440399169922, mu_star: 5.67012882232666
Epoch 7500, Loss: 29.92434310913086, 

In [25]:
# Plot the estimated mu_star and in a separate plot the estimated solution
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=mus, mode='lines', name='Estimated mu_star', line=dict(color='blue')))
fig.add_shape(type='line', x0=0, y0=mu, x1=epochs, y1=mu, line=dict(color='red', dash='dash'))
fig.update_layout(title='Estimated damping coefficient', xaxis_title='Epoch', yaxis_title='mu_star')
fig.update_layout(template='plotly_dark')
fig.show()

last_results = results[-1]
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_train.detach().numpy().flatten(), y=last_results.flatten(), mode='lines', name='PINN solution', line=dict(color='blue', width=10)))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=torch.zeros_like(t_test).flatten(), mode='lines', name='Equilibrium position', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(x=t_test.flatten(), y=exact_solution(delta, w0, t_test).flatten(), mode='lines', name='Exact solution', line=dict(color='lightblue')))
fig.add_trace(go.Scatter(x=t_data.flatten(), y=x_data.flatten(), mode='markers', name='Data points', marker=dict(size=10, color='lightblue')))
fig.update_layout(title='PINN solution of the under-damped harmonic oscillator', xaxis_title='Time', yaxis_title='Displacement')
fig.update_layout(template='plotly_dark')
fig.update_yaxes(range=[-1,1])
fig.update_xaxes(range=[0,1])
fig.show()

# R² score
print('R² =', r2_score(exact_solution(delta, w0, t_train.detach()).flatten(), last_results.flatten()))

R² = 0.9980765361995401


# THE END
If you are interested in the continuation of this notebook, please let me know and it will be available on github :)